### Lab 8.1 Tokenization

In this lab you will explore different methods of text tokenization.  

The following code will download the text of Shakespeare's sonnets and read it in as one long string.

In [1]:
from torch.utils.data import Dataset

In [2]:
!wget --no-clobber "https://www.dropbox.com/scl/fi/7r68l64ijemidyb9lf80q/sonnets.txt?rlkey=udb47coatr2zbrk31hsfbr22y&dl=1" -O sonnets.txt
text = (open("sonnets.txt").read())


--2025-02-25 04:43:44--  https://www.dropbox.com/scl/fi/7r68l64ijemidyb9lf80q/sonnets.txt?rlkey=udb47coatr2zbrk31hsfbr22y&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc42f7dccf91b538449f315cda16.dl.dropboxusercontent.com/cd/0/inline/CkzF72MfElOg_kY-xFMBv_xBkcHgtbdvxdU3j32Sflfyu-ICTQPbH9yAgz1gbomAfl6Y2tIvvVOHH8gu9UoeDlaKq6goW2orDtHWbCToY9Cn-2_6KhvJfbAg0-j6iyARg1fNKfxB3w3JlzKOQS3u29gn/file?dl=1# [following]
--2025-02-25 04:43:45--  https://uc42f7dccf91b538449f315cda16.dl.dropboxusercontent.com/cd/0/inline/CkzF72MfElOg_kY-xFMBv_xBkcHgtbdvxdU3j32Sflfyu-ICTQPbH9yAgz1gbomAfl6Y2tIvvVOHH8gu9UoeDlaKq6goW2orDtHWbCToY9Cn-2_6KhvJfbAg0-j6iyARg1fNKfxB3w3JlzKOQS3u29gn/file?dl=1
Resolving uc42f7dccf91b538449f315cda16.dl.dropboxusercontent.com (uc42f7dccf91b538449f315cda16.dl.dropboxusercontent.com)... 162.12

In [3]:
text = text.lower()

In [4]:
print(text[:1000])

i

 from fairest creatures we desire increase,
 that thereby beauty's rose might never die,
 but as the riper should by time decease,
 his tender heir might bear his memory:
 but thou, contracted to thine own bright eyes,
 feed'st thy light's flame with self-substantial fuel,
 making a famine where abundance lies,
 thy self thy foe, to thy sweet self too cruel:
 thou that art now the world's fresh ornament,
 and only herald to the gaudy spring,
 within thine own bud buriest thy content,
 and tender churl mak'st waste in niggarding:
   pity the world, or else this glutton be,
   to eat the world's due, by the grave and thee.

 ii

 when forty winters shall besiege thy brow,
 and dig deep trenches in thy beauty's field,
 thy youth's proud livery so gazed on now,
 will be a tatter'd weed of small worth held:
 then being asked, where all thy beauty lies,
 where all the treasure of thy lusty days;
 to say, within thine own deep sunken eyes,
 were an all-eating shame, and thriftless praise.


### Exercises

1. Prepare a vocabulary of the unique words in the dataset.  (For simplicity's sake you can leave the punctuation in.)

In [5]:
vocab_words = set(text.split())

2. Now you will make a Dataset subclass that can return sequences of tokens, encoded as integers.

In [6]:
class WordDataset(Dataset):
  def __init__(self,text,seq_len=100):
    self.seq_len = seq_len
    self.vocab = set(text.split())
    self.vocab_size = len(self.vocab)
    self.word2idx = {word: idx for idx, word in enumerate(self.vocab)}
    self.indices = [self.word2idx[word] for word in text.split()]
    # add code to compute the vocabulary (copied from exercise 1)
    # add code to convert the text to a sequence of word indices

  def __len__(self):
    # replace this with code to return the number of possible sub-sequences
    return len(self.indices) - self.seq_len

  def __getitem__(self,i):
    # replace this with code to return the sequence of token indices starting at i and the index of token i+seq_len as the label
    return self.indices[i:i+self.seq_len], self.indices[i+self.seq_len]

  def decode(self,tokens):
    # replace this with code to convert a sequence of tokens back into a string
    return ' '.join([list(self.vocab)[token] for token in tokens])


3. Verify that your class can successfully encode and decode sequences.

In [15]:
dataset_word = WordDataset(text)
coded_sequence, label = dataset_word.__getitem__(0)
print(coded_sequence)
print(dataset_word.decode(coded_sequence))


[2435, 3715, 1749, 1450, 2913, 1870, 2518, 972, 2352, 3517, 1169, 3444, 764, 4479, 1803, 408, 733, 2260, 2274, 956, 3908, 478, 71, 4007, 3843, 3444, 1393, 71, 3634, 1803, 2463, 3569, 375, 3665, 3982, 4581, 3531, 557, 1469, 2824, 4182, 1370, 1901, 3838, 2709, 859, 4503, 745, 4273, 4238, 1469, 4528, 1469, 760, 375, 1469, 3465, 4528, 1162, 792, 1821, 972, 820, 4013, 733, 3000, 4027, 1984, 3382, 1426, 1584, 375, 733, 4571, 3484, 790, 3665, 3982, 4186, 2768, 1469, 4537, 3382, 4007, 3844, 3888, 4329, 1678, 101, 91, 733, 1166, 1569, 3212, 24, 3722, 3145, 375, 2046, 733]
﻿i from fairest creatures we desire increase, that thereby beauty's rose might never die, but as the riper should by time decease, his tender heir might bear his memory: but thou, contracted to thine own bright eyes, feed'st thy light's flame with self-substantial fuel, making a famine where abundance lies, thy self thy foe, to thy sweet self too cruel: thou that art now the world's fresh ornament, and only herald to the gaudy

4. Do the exercise again, but this time at the character level.

In [23]:
class CharacterDataset(Dataset):
  def __init__(self,text,seq_len=100):
    self.seq_len = seq_len
    self.vocab = set(text)
    self.vocab_size = len(self.vocab)
    self.char2idx = {char: idx for idx, char in enumerate(self.vocab)}
    self.indices = [self.char2idx[char] for char in text]
    # add code to compute the vocabulary of unique characters
    # add code to convert the text to a sequence of character indices

  def __len__(self):
    # replace this with code to return the number of possible sub-sequences
    return len(self.indices) - self.seq_len

  def __getitem__(self,i):
    # replace this with code to return the sequence of token indices starting at i and the index of token i+seq_len as the label
    return self.indices[i:i+self.seq_len], self.indices[i+self.seq_len]

  def decode(self,tokens):
    # replace this with code to convert a sequence of tokens back into a string
    return ''.join([list(self.vocab)[token] for token in tokens])

In [24]:
dataset_char = CharacterDataset(text)
coded_sequence, label = dataset_char.__getitem__(0)
print(coded_sequence)
print(dataset_char.decode(coded_sequence))

[9, 13, 21, 21, 26, 3, 17, 24, 19, 26, 3, 10, 13, 17, 15, 5, 27, 26, 29, 17, 15, 10, 27, 23, 17, 15, 5, 26, 1, 15, 26, 16, 15, 5, 13, 17, 15, 26, 13, 0, 29, 17, 15, 10, 5, 15, 28, 21, 26, 27, 8, 10, 27, 26, 27, 8, 15, 17, 15, 20, 22, 26, 20, 15, 10, 23, 27, 22, 31, 5, 26, 17, 24, 5, 15, 26, 19, 13, 12, 8, 27, 26, 0, 15, 25, 15, 17, 26, 16, 13, 15, 28, 21, 26, 20, 23, 27, 26, 10, 5]
﻿i

 from fairest creatures we desire increase,
 that thereby beauty's rose might never die,
 but as


5. Compare the number of sequences for each tokenization method.

In [25]:
print(f'Number of sequences for word tokenization: {dataset_word.__len__()}')
print(f'Number of sequences for character tokenization: {dataset_char.__len__()}')

Number of sequences for word tokenization: 17570
Number of sequences for character tokenization: 97820


6. Optional: implement the byte pair encoding algorithm to make a Dataset class that uses word parts.

### Summary of Lab 8

Lab 8 focused on exploring different methods of text tokenization using Shakespeare's sonnets. The lab included exercises on creating vocabulary from the text, implementing Dataset subclasses for word and character tokenization, encoding and decoding sequences, and comparing the number of sequences generated by each method. Optional exercises included implementing byte pair encoding.